In [8]:
import sys
import os

root_dir = os.path.split(os.getcwd())[0]

sys.path.append(root_dir)
from utils.helper_metastore import *
from utils.configurations.config import Config

In [10]:
import warnings
warnings.filterwarnings('ignore', 'absl')

%load_ext tensorboard

## Model Training

Here comes the most important part of our pipeline (model training). The whole pipeline that we are trying to build on are jointly called as an Continous training component in the MLOps workflow. The main moto over here is to keep our model upto date in the production and the counter back the data drift, model drift and training-serving skew.

![mlops pipeline](image/MLOps_pipeline.svg)

In this notebook, we cover the model training process as part of a machine learning pipeline, including how it is automated in a TFX pipeline. We also include some details of distribution strategies available in TensorFlow and how to tune hyperparameters in a pipeline. This chapter is more specific to TFX pipelines than most of the
others because we don’t cover training as a standalone process.

One very important feature of training a model in a TFX pipeline is that the data pre-processing step which ar saved along with the trained model weights. This is very useful once our model is deployed to production because it means that the preprocessing steps will always produce the features the models expecting. Without this feature, it would be possible to update the data pre‐processing steps without updating the model, and then the model would fail in production. So why TFX eports the proprocessing steps and the model as one graph, this potentialy eliminates the source of error.

## Defining the model

In this session, let assume we create some model which was trained as offline experimentation. we then decided to productionize it with the MLOps worflow. In this pipeline step, we want to export the model with our preprocessing steps, we need to guarantee that the model input names match the transformed feature names from ```preprocessing_fn()```. In our example model, we reuse the ```transformed_name()``` function to add the suffix _xf to our features.

In [3]:
%%bash

mkdir ../models

In [1]:
import tensorflow as tf
import tensorflow_hub as hub


def transformed_name(key):
    return key + '_xf'


def get_model():
    LABEL_KEY = "consumer_disputed"
    # Loop over the features and create an input for each feature.
    # Feature name, feature dimensionality.
    ONE_HOT_FEATURES = {
    "product": 11,
    "sub_product": 45,
    "company_response": 5,
    "state": 60,
    "issue": 90
    }

    # Feature name, bucket count.
    BUCKET_FEATURES = {
    "zip_code": 10
    }

    # Feature name, value is unused.
    TEXT_FEATURES = {
    "consumer_complaint_narrative": None
    }
    # One-hot categorical features
    input_features = []
    for key, dim in ONE_HOT_FEATURES.items():
        input_features.append(
            tf.keras.Input(shape=(dim + 1,),
            name=transformed_name(key)))
        
    # Adding bucketized features
    for key, dim in BUCKET_FEATURES.items():
        input_features.append(
            tf.keras.Input(shape=(dim + 1,),
            name=transformed_name(key)))
    
    # Adding text input features
    input_texts = []
    for key in TEXT_FEATURES.keys():
        input_texts.append(
            tf.keras.Input(shape=(1,),
            name=transformed_name(key),
            dtype=tf.string))
    
    inputs = input_features + input_texts
    # Embed text features
    # Load the tf.hub module of the Universal Sentence Encoder model.
    MODULE_URL = 
    embed = hub.KerasLayer(MODULE_URL)
    
    reshaped_narrative = tf.reshape(input_texts[0], [-1]) # Keras inputs are two-dimensional, 
                                                            # but the encoder expects one-dimensional inputs.
    embed_narrative = embed(reshaped_narrative)
    deep_ff = tf.keras.layers.Reshape((512, ), input_shape=(1, 512))(embed_narrative)
    deep = tf.keras.layers.Dense(256, activation='relu')(deep_ff)
    deep = tf.keras.layers.Dense(64, activation='relu')(deep)
    deep = tf.keras.layers.Dense(16, activation='relu')(deep)
    wide_ff = tf.keras.layers.concatenate(input_features)
    wide = tf.keras.layers.Dense(16, activation='relu')(wide_ff)
    both = tf.keras.layers.concatenate([deep, wide])
    output = tf.keras.layers.Dense(1, activation='sigmoid')(both)
    keras_model = tf.keras.models.Model(inputs, output)
    keras_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=[
    tf.keras.metrics.BinaryAccuracy(),
    tf.keras.metrics.TruePositives()
    ])
    return keras_model

In [5]:
# model = get_model()

embedding = hub.KerasLayer('../models/universal-sentence-encoder_4')

<tf.Tensor: shape=(1, 512), dtype=float32, numpy=
array([[-5.73915504e-02, -1.82517227e-02,  5.40610440e-02,
        -1.05629796e-02,  5.26848733e-02, -1.70335826e-02,
         4.58819084e-02, -6.38231784e-02,  1.02628479e-02,
         5.89594841e-02,  3.36269476e-02,  2.78246552e-02,
        -5.38831130e-02,  8.77926778e-03,  2.70789620e-02,
        -7.52516165e-02, -4.78043370e-02,  1.69670079e-02,
        -4.39201556e-02, -6.64973035e-02,  9.78493690e-02,
        -2.02894080e-02,  3.17293359e-03,  1.30768614e-02,
        -6.03378750e-02,  2.25895438e-02,  4.08571921e-02,
         5.44126444e-02, -2.88982093e-02,  3.93695123e-02,
         5.40930815e-02,  1.13724852e-02,  3.97175103e-02,
        -1.39451139e-02, -3.05868089e-02, -1.02045918e-02,
        -5.70705626e-04, -3.76856588e-02,  6.99091610e-03,
        -1.82859716e-03,  3.30865309e-02,  2.19007153e-02,
         2.42540464e-02,  5.49291223e-02,  4.06780802e-02,
        -7.57424568e-05, -9.26447138e-02,  2.11828724e-02,
      

In [ ]:
tf.keras.utils.plot_model(model, show_shapes = True, show_layer_names = True)